### Symantecのattack signatureページを読み込んで見る

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup 
import re

html = urlopen('https://www.symantec.com/security_response/attacksignatures/')
bs = BeautifulSoup(html, 'html.parser')

In [ ]:
#URLのリストを作成する
def makeURL(link):
    return 'https://www.symantec.com'+link

links = bs.find('div', {'class': 'bckPadLG bckSolidWht unit'}).findAll('a',href=re.compile('^(/security_response/)'))
URLs=[makeURL(l.attrs['href']) for l in links]
len(URLs)

In [ ]:
def get_attack_details(link):
    '''
    return title and bs object
    '''
    html = urlopen(link)
    bs= BeautifulSoup(html, 'html.parser') 
    return {'name':bs.h1,
            'contents':bs.find('div', {'class':'bckPadLG bckSolidWht unit'})}

# get_attack_details(URLs[0])
contents=[get_attack_details(l) for l in URLs]

しかしここで、問題が発生した。exploit dateが見当たらないことである。論文中では活用されていたのになぜ...